In [2]:
!pip install geopandas
!pip install fiona
!pip install pandas
!pip install pyproj
!pip install os
!pip install shapely
!pip install unary_union

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement unary_union (from versions: none)
ERROR: No matching distribution found for unary_union


In [3]:
import geopandas as gpd
import fiona
import pandas as pd
import pyproj
import os
from shapely.geometry import shape
from shapely.ops import split, unary_union

In [4]:
import time

def time_it(func):
    """
    A decorator function to measure the execution time of a given function.

    Args:
        func: The function whose execution time needs to be measured.

    Returns:
        A wrapped version of the input function that prints the execution time.
    """
    def wrapper(*args, **kwargs):
        # Record the start time
        start_time = time.time()

        # Call the original function with its arguments
        result = func(*args, **kwargs)

        # Record the end time
        end_time = time.time()

        # Calculate and print the elapsed time
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' took {elapsed_time:.6f} seconds to execute.")

        # Return the result of the original function
        return result

    return wrapper


In [5]:
pd.set_option('display.max_columns', None)


In [10]:
muni_df = gpd.read_file("MunicipalitiesRegion.tab", driver='MapInfo File') # Explicitly specify the MapInfo File driver

In [11]:
muni_df = muni_df[muni_df['PROV']=="AB"].sort_values('NAME')

In [12]:
model_area_lut = muni_df[['UNIQUE_ID','NAME','PROV','geometry']]
model_area_lut.shape

(425, 4)

In [16]:
model_areas = gpd.read_file("/ModelAreas_AB.TAB", driver='MapInfo File')

In [17]:
model_area_lut['Obj_Npolygons'] = model_area_lut.geometry.apply(lambda g: 1 if g.geom_type == "Polygon" else len(g.geoms))



/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [18]:
df = model_area_lut[model_area_lut['Obj_Npolygons']>1]

In [20]:
df.shape

(16, 5)